<a href="https://colab.research.google.com/github/Adrija-debug/Conv-Tasnet/blob/master/MixingSpeechMusic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Adrija-debug/MS-SNSD.git

Cloning into 'MS-SNSD'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 29918 (delta 50), reused 87 (delta 16), pack-reused 29779
Receiving objects: 100% (29918/29918), 3.93 GiB | 31.12 MiB/s, done.
Resolving deltas: 100% (74/74), done.
Checking out files: 100% (24399/24399), done.


In [2]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.5).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 1s (179 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 145480 files and directories currently installed.)
Preparing to 

In [3]:
!pip install numpy glob2 soundfile scipy configparser audiolib

  Created wheel for audiolib: filename=AudioLib-1.0.0b1-cp36-none-any.whl size=13177 sha256=9697f4fd479a2ff71cfb7910309445d5ed5d09095711319ddad8edbc8eb5b5dd
  Stored in directory: /root/.cache/pip/wheels/4c/f7/85/2e323ceb106db9a9bdb55f66f29444b16166dd99b36abaf19e
  Created wheel for pyaudio: filename=PyAudio-0.2.11-cp36-cp36m-linux_x86_64.whl size=51615 sha256=3c335111bb78d851381da5956293823a34c3bbdad90ccf0985a56095083bc6ec
  Stored in directory: /root/.cache/pip/wheels/f4/a8/a4/292214166c2917890f85b2f72a8e5f13e1ffa527c4200dcede
Successfully built audiolib pyaudio


In [4]:
# Not to be edited...
import soundfile as sf
import os
import numpy as np

# Function to read audio
def audioread(path, norm = True, start=0, stop=None):
    path = os.path.abspath(path)
    if not os.path.exists(path):
        raise ValueError("[{}] does not exist!".format(path))
    try:
        x, sr = sf.read(path, start=start, stop=stop)
    except RuntimeError:  # fix for sph pcm-embedded shortened v2
        print('WARNING: Audio type not supported')

    if len(x.shape) == 1:  # mono
        if norm:
            rms = (x ** 2).mean() ** 0.5
            scalar = 10 ** (-25 / 20) / (rms)
            x = x * scalar
        return x, sr
    else:  # multi-channel
        x = x.T
        x = x.sum(axis=0)/x.shape[0]
        if norm:
            rms = (x ** 2).mean() ** 0.5
            scalar = 10 ** (-25 / 20) / (rms)
            x = x * scalar
        return x, sr
    
# Funtion to write audio    
def audiowrite(data, fs, destpath, norm=False):
    if norm:
        rms = (data ** 2).mean() ** 0.5
        scalar = 10 ** (-25 / 10) / (rms+eps)
        data = data * scalar
        if max(abs(data))>=1:
            data = data/max(abs(data), eps)
    
    destpath = os.path.abspath(destpath)
    destdir = os.path.dirname(destpath)
    
    if not os.path.exists(destdir):
        os.makedirs(destdir)
    
    sf.write(destpath, data, fs)
    return

# Function to mix clean speech and music at various SNR levels
def snr_mixer(clean, noise, snr):
    # Normalizing to -25 dB FS
    rmsclean = (clean**2).mean()**0.5
    scalarclean = 10 ** (-25 / 20) / rmsclean
    clean = clean * scalarclean
    rmsclean = (clean**2).mean()**0.5

    rmsnoise = (noise**2).mean()**0.5
    scalarnoise = 10 ** (-25 / 20) /rmsnoise
    noise = noise * scalarnoise
    rmsnoise = (noise**2).mean()**0.5
    
    # Set the noise level for a given SNR
    noisescalar = np.sqrt(rmsclean / (10**(snr/20)) / rmsnoise)
    noisenewlevel = noise * noisescalar
    noisyspeech = clean + noisenewlevel
    return clean, noisenewlevel, noisyspeech
        
    


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os.path
path = os.path.join(os.getcwd(), 'drive','MyDrive','MusicFiles')
num_files = len([f for f in os.listdir(path)if os.path.isfile(os.path.join(path, f))])
print(num_files)

660


In [7]:
import os.path
path = os.path.join(os.getcwd(), 'drive','MyDrive','SpeechFiles')
num_files = len([f for f in os.listdir(path)if os.path.isfile(os.path.join(path, f))])
print(num_files)

426


In [8]:
# mixing speech and music
import glob
import numpy as np
import soundfile as sf
import os
import argparse
import configparser as CP
# from audiolib import audioread, audiowrite, snr_mixer

def main(cfg):
    snr_lower = int(cfg["snr_lower"])
    snr_upper = int(cfg["snr_upper"])
    total_snrlevels = int(cfg["total_snrlevels"])
    
    clean_dir = os.path.join(os.getcwd(), 'drive','MyDrive','SpeechFiles')
    #speech
    if cfg["speech_dir"]!='None':
        clean_dir = cfg["speech_dir"]
    if not os.path.exists(clean_dir):
        assert False, ("Clean speech data is required")
    
    noise_dir = os.path.join(os.getcwd(), 'drive','MyDrive','MusicFiles')
    #music
    if cfg["noise_dir"]!='None':
        noise_dir = cfg["noise_dir"]
    if not os.path.exists(noise_dir):
        assert False, ("Noise data is required")
        
    fs = float(cfg["sampling_rate"])
    audioformat = cfg["audioformat"]
    total_hours = float(cfg["total_hours"])
    audio_length = float(cfg["audio_length"])
    silence_length = float(cfg["silence_length"])
    noisyspeech_dir = os.path.join(os.getcwd(), 'mix')
    if not os.path.exists(noisyspeech_dir):
        os.makedirs(noisyspeech_dir)
    clean_proc_dir = os.path.join(os.getcwd(), 's1')
    if not os.path.exists(clean_proc_dir):
        os.makedirs(clean_proc_dir)
    noise_proc_dir = os.path.join(os.getcwd(), 's2')
    if not os.path.exists(noise_proc_dir):
        os.makedirs(noise_proc_dir)
        
    total_secs = total_hours*60*60
    total_samples = int(total_secs * fs)
    audio_length = int(audio_length*fs)
    SNR = np.linspace(snr_lower, snr_upper, total_snrlevels)
    cleanfilenames = glob.glob(os.path.join(clean_dir, audioformat))
    if cfg["noise_types_excluded"]=='None':
        noisefilenames = glob.glob(os.path.join(noise_dir, audioformat))
    else:
        filestoexclude = cfg["noise_types_excluded"].split(',')
        noisefilenames = glob.glob(os.path.join(noise_dir, audioformat))
        for i in range(len(filestoexclude)):
            noisefilenames = [fn for fn in noisefilenames if not os.path.basename(fn).startswith(filestoexclude[i])]
    
    filecounter = 0
    num_samples = 0
    
    while num_samples < total_samples:
        idx_s = np.random.randint(0, np.size(cleanfilenames))
        clean, fs = audioread(cleanfilenames[idx_s])
        
        if len(clean) == audio_length:
            clean = clean
        
        else:
            
            while len(clean)<=audio_length:
                idx_s = idx_s + 1
                if idx_s >= np.size(cleanfilenames)-1:
                    idx_s = np.random.randint(0, np.size(cleanfilenames)) 
                newclean, fs = audioread(cleanfilenames[idx_s])
                cleanconcat = np.append(clean, np.zeros(int(fs*silence_length)))
                clean = np.append(cleanconcat, newclean)
        clean = clean[0:audio_length]
        idx_n = np.random.randint(0, np.size(noisefilenames))
        noise, fs = audioread(noisefilenames[idx_n])
        
        if len(noise)>=len(clean):
            noise = noise[0:len(clean)]
        
        else:
        
            while len(noise)<=len(clean):
                idx_n = idx_n + 1
                if idx_n >= np.size(noisefilenames)-1:
                    idx_n = np.random.randint(0, np.size(noisefilenames))
                newnoise, fs = audioread(noisefilenames[idx_n])
                noiseconcat = np.append(noise, np.zeros(int(fs*silence_length)))
                noise = np.append(noiseconcat, newnoise)
        noise = noise[0:len(clean)]
        filecounter = filecounter + 1
       
        for i in range(np.size(SNR)):
            clean_snr, noise_snr, noisy_snr = snr_mixer(clean=clean, noise=noise, snr=SNR[i])
            noisyfilename = 'music'+str(filecounter)+'_SNRdb_'+str(SNR[i])+'_clnsp'+str(filecounter)+'.wav'
            cleanfilename = 'clean'+str(filecounter)+'_.wav'
            noisefilename = 'music'+str(filecounter)+'_SNRdb_'+str(SNR[i])+'.wav'
            noisypath = os.path.join(noisyspeech_dir, noisyfilename)
            cleanpath = os.path.join(clean_proc_dir, cleanfilename)
            noisepath = os.path.join(noise_proc_dir, noisefilename)
            audiowrite(noisy_snr, fs, noisypath, norm=False)
            audiowrite(clean_snr, fs, cleanpath, norm=False)
            audiowrite(noise_snr, fs, noisepath, norm=False)
            num_samples = num_samples + len(noisy_snr)
            

if __name__=="__main__":
    # parser = argparse.ArgumentParser()
    
    # # Configurations: read noisyspeech_synthesizer.cfg
    # parser.add_argument("--cfg", default = "noisyspeech_synthesizer.cfg", help = "Read noisyspeech_synthesizer.cfg for all the details")
    # parser.add_argument("--cfg_str", type=str, default = "noisy_speech" )
    # args = parser.parse_args()

    
    cfgpath = os.path.join(os.getcwd(), 'MS-SNSD', 'noisyspeech_synthesizer.cfg')
    assert os.path.exists(cfgpath), f"No configuration file as [{cfgpath}]"
    cfg = CP.ConfigParser()
    cfg._interpolation = CP.ExtendedInterpolation()
    cfg.read(cfgpath)
    
    main(cfg._sections['noisy_speech'])
    input("Mixed successfully.\nPress any key to continue.\n")


Mixed successfully.
Press any key to continue.



In [10]:
import os.path
path = os.path.join(os.getcwd(), 'mix')
num_files = len([f for f in os.listdir(path)if os.path.isfile(os.path.join(path, f))])
print(num_files)

1080


In [11]:
import os.path
path = os.path.join(os.getcwd(), 's1')
num_files = len([f for f in os.listdir(path)if os.path.isfile(os.path.join(path, f))])
print(num_files)

1080


In [12]:
import os.path
path = os.path.join(os.getcwd(), 's2')
num_files = len([f for f in os.listdir(path)if os.path.isfile(os.path.join(path, f))])
print(num_files)

1080


In [13]:
!git clone https://github.com/joshijai2/Conv-Tasnet

Cloning into 'Conv-Tasnet'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 134 (delta 67), reused 80 (delta 32), pack-reused 0
Receiving objects: 100% (134/134), 19.19 MiB | 25.62 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [14]:
!pwd

/content


In [15]:
!dir

Conv-Tasnet  drive  mix  MS-SNSD  s1  s2  sample_data


In [16]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.5MB 101kB/s 
     |████████████████████████████████| 368kB 48.7MB/s 
     |████████████████████████████████| 3.2MB 39.2MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: tensorflow 2.4.0
    Uninstalling tensorflow-2.4.0:
      Successfully uninstalled tensorflow-2.4.0


In [22]:
!zip -r /content/s1 /content/Conv-Tasnet/data/train/s1

  adding: content/Conv-Tasnet/data/train/s1/ (stored 0%)
  adding: content/Conv-Tasnet/data/train/s1/clean523_.wav (deflated 23%)
  adding: content/Conv-Tasnet/data/train/s1/clean106_.wav (deflated 13%)
  adding: content/Conv-Tasnet/data/train/s1/clean50_.wav (deflated 19%)
  adding: content/Conv-Tasnet/data/train/s1/clean877_.wav (deflated 14%)
  adding: content/Conv-Tasnet/data/train/s1/clean299_.wav (deflated 36%)
  adding: content/Conv-Tasnet/data/train/s1/clean707_.wav (deflated 16%)
  adding: content/Conv-Tasnet/data/train/s1/clean1028_.wav (deflated 17%)
  adding: content/Conv-Tasnet/data/train/s1/clean532_.wav (deflated 19%)
  adding: content/Conv-Tasnet/data/train/s1/clean73_.wav (deflated 12%)
  adding: content/Conv-Tasnet/data/train/s1/clean47_.wav (deflated 13%)
  adding: content/Conv-Tasnet/data/train/s1/clean305_.wav (deflated 24%)
  adding: content/Conv-Tasnet/data/train/s1/clean830_.wav (deflated 12%)
  adding: content/Conv-Tasnet/data/train/s1/clean348_.wav (deflated 2

In [23]:
!zip -r /content/s2 /content/Conv-Tasnet/data/train/s2

  adding: content/Conv-Tasnet/data/train/s2/ (stored 0%)
  adding: content/Conv-Tasnet/data/train/s2/music760_SNRdb_30.0.wav (deflated 26%)
  adding: content/Conv-Tasnet/data/train/s2/music906_SNRdb_30.0.wav (deflated 29%)
  adding: content/Conv-Tasnet/data/train/s2/music507_SNRdb_30.0.wav (deflated 30%)
  adding: content/Conv-Tasnet/data/train/s2/music145_SNRdb_30.0.wav (deflated 23%)
  adding: content/Conv-Tasnet/data/train/s2/music268_SNRdb_30.0.wav (deflated 30%)
  adding: content/Conv-Tasnet/data/train/s2/music375_SNRdb_30.0.wav (deflated 27%)
  adding: content/Conv-Tasnet/data/train/s2/music581_SNRdb_30.0.wav (deflated 27%)
  adding: content/Conv-Tasnet/data/train/s2/music124_SNRdb_30.0.wav (deflated 26%)
  adding: content/Conv-Tasnet/data/train/s2/music29_SNRdb_30.0.wav (deflated 57%)
  adding: content/Conv-Tasnet/data/train/s2/music989_SNRdb_30.0.wav (deflated 37%)
  adding: content/Conv-Tasnet/data/train/s2/music396_SNRdb_30.0.wav (deflated 26%)
  adding: content/Conv-Tasnet/d

In [24]:
!zip -r /content/mix /content/Conv-Tasnet/data/train/mix

  adding: content/Conv-Tasnet/data/train/mix/ (stored 0%)
  adding: content/Conv-Tasnet/data/train/mix/music872_SNRdb_30.0_clnsp872.wav (deflated 16%)
  adding: content/Conv-Tasnet/data/train/mix/music774_SNRdb_30.0_clnsp774.wav (deflated 16%)
  adding: content/Conv-Tasnet/data/train/mix/music510_SNRdb_30.0_clnsp510.wav (deflated 14%)
  adding: content/Conv-Tasnet/data/train/mix/music606_SNRdb_30.0_clnsp606.wav (deflated 17%)
  adding: content/Conv-Tasnet/data/train/mix/music827_SNRdb_30.0_clnsp827.wav (deflated 15%)
  adding: content/Conv-Tasnet/data/train/mix/music227_SNRdb_30.0_clnsp227.wav (deflated 18%)
  adding: content/Conv-Tasnet/data/train/mix/music561_SNRdb_30.0_clnsp561.wav (deflated 15%)
  adding: content/Conv-Tasnet/data/train/mix/music219_SNRdb_30.0_clnsp219.wav (deflated 12%)
  adding: content/Conv-Tasnet/data/train/mix/music603_SNRdb_30.0_clnsp603.wav (deflated 16%)
  adding: content/Conv-Tasnet/data/train/mix/music111_SNRdb_30.0_clnsp111.wav (deflated 15%)
  adding: co

In [28]:
!unzip "/content/s1.zip" -d "/content/Conv-Tasnet/data/infer"

Archive:  /content/s1.zip
   creating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean523_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean106_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean50_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean877_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean299_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean707_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean1028_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean532_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean73_.wav  
  inflating: /content/Conv-Tasnet/data/i

In [ ]:
!unzip "/content/s1.zip" -d "/content/Conv-Tasnet/data/infer"

Archive:  /content/s1.zip
   creating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean50_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean73_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean47_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean14_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean12_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean71_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean85_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean32_.wav  
  inflating: /content/Conv-Tasnet/data/infer/content/Conv-Tasnet/data/train/s1/clean80_.wav  
  inflating: /content/Conv-Tasnet/data/infer/con

In [ ]:
!python -W ignore ./Conv-Tasnet/main.py

Instructions for updating:
Colocations handled automatically by placer.
From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Writing /content/Conv-Tasnet/data/train.tfr
1080
100% 1080/1080 [08:40<00:00,  2.08it/s]
Writing /content/Conv-Tasnet/data/valid.tfr
1080
100% 1080/1080 [08:45<00:00,  2.06it/s]
Loading data from /content/Conv-Tasnet/data/train.tfr
Loading data from /content/Conv-Tasnet/data/valid.tfr
Instructions for updating:
Use tf.cast instead.
From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
# of trainable parameters: 172224
Loading model from /content/Conv-Tasnet/logs/model.

In [ ]:
!python ./Conv-Tasnet/main.py --mode test 

In [ ]:
!python ./Conv-Tasnet/main.py --mode test 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 